In [1]:
from langchain_community.utilities import SQLDatabase
from pyprojroot import here
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

True

Connecting to the sqldb

In [3]:
db_path = r"data\sql\bank_domain.db"
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")
db

In [5]:
# validate the connection to the vectordb
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM accounts LIMIT 10;")

sqlite
['accounts', 'branch_locations', 'credit_cards', 'credit_exposures', 'customers', 'employees', 'insurance_policies', 'investments', 'loans', 'transactions']


"[(1, 33, 'Checking', 9155.54, '2023-03-19'), (2, 94, 'Money Market', 24755.42, '2015-03-18'), (3, 17, 'Money Market', 19345.75, '2019-09-28'), (4, 92, 'Money Market', 5670.63, '2022-02-07'), (5, 82, 'Savings', 18791.14, '2015-01-05'), (6, 46, 'Savings', 27208.22, '2024-07-16'), (7, 87, 'Savings', 27590.0, '2022-02-24'), (8, 100, 'Checking', 3130.15, '2016-11-20'), (9, 74, 'Savings', 20973.09, '2022-08-09'), (10, 18, 'Savings', 23597.94, '2020-12-30')]"

In [8]:
from langchain_groq import ChatGroq
# llm=ChatGroq(model="llama3-70b-8192")
llm = ChatGroq(
    temperature = 0,
    groq_api_key = os.getenv("groq_api_key"),
    model_name = os.getenv("llama_model_name1")
)

In [9]:
llm.invoke("hello how are you?")

AIMessage(content="Hello! I'm just a language model, so I don't have feelings or emotions like humans do. However, I'm functioning properly and ready to assist you with any questions or tasks you may have. How can I help you today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 15, 'total_tokens': 64, 'completion_time': 0.143990343, 'prompt_time': 0.00334006, 'queue_time': 0.019602879, 'total_time': 0.147330403}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_753a4aecf6', 'finish_reason': 'stop', 'logprobs': None}, id='run-4dc078b0-92d2-406c-8394-8cd2de432387-0', usage_metadata={'input_tokens': 15, 'output_tokens': 49, 'total_tokens': 64})

#### 1. SQL query chain

In [ ]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many employees are there"})
print(response)

In [ ]:
db.run("SELECT COUNT('EmployeeId') FROM 'Employee';")

In [ ]:
chain.get_prompts()[0].pretty_print()

#### Add QuerySQLDataBaseTool to the chain

In [ ]:
from langchain_community.tools import QuerySQLDataBaseTool

write_query = create_sql_query_chain(llm, db)
execute_query = QuerySQLDataBaseTool(db=db)

chain = write_query | execute_query

chain.invoke({"question": "How many employees are there"})

#### Answer the question in a user friendly manner

In [ ]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

chain.invoke({"question": "How many employees are there"})

#### 2. Agents

In [10]:
from langchain_community.agent_toolkits import create_sql_agent

agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", 
                                  verbose=True,
                                  agent_executor_kwargs = {"return_intermediate_steps": True}
                                  )

In [15]:
response = agent_executor.invoke(
    {
        "input": "What is the average balance of accounts opened in the last six months, categorized by account type and broken down by the state of the branch the associated customer is linked to, via employee records?"
    }
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{'tool_input': ''}`
responded:   Let me do that.



accounts, branch_locations, credit_cards, credit_exposures, customers, employees, insurance_policies, investments, loans, transactions
Invoking: `sql_db_schema` with `{'table_names': 'accounts, customers, employees, branch_locations'}`



CREATE TABLE accounts (
	account_id INTEGER, 
	customer_id INTEGER, 
	account_type TEXT, 
	balance REAL, 
	open_date DATE, 
	PRIMARY KEY (account_id), 
	FOREIGN KEY(customer_id) REFERENCES customers (customer_id)
)

/*
3 rows from accounts table:
account_id	customer_id	account_type	balance	open_date
1	33	Checking	9155.54	2023-03-19
2	94	Money Market	24755.42	2015-03-18
3	17	Money Market	19345.75	2019-09-28
*/


CREATE TABLE branch_locations (
	branch_id INTEGER, 
	branch_name TEXT, 
	address TEXT, 
	city TEXT, 
	state TEXT, 
	zip_code TEXT, 
	PRIMARY KEY (branch_id)
)

/*
3 rows from branch_locations table:
branch_id	b

In [16]:
queries = []
for (log, output) in response["intermediate_steps"]:
    if log.tool == 'sql_db_query':
        queries.append(log.tool_input)

In [17]:
print(f"Query\n{queries[-1]['query']}\nAnswer\n{response['output']}")

Query
SELECT account_type, bl.state, AVG(balance) AS average_balance FROM accounts a JOIN customers c ON a.customer_id = c.customer_id JOIN employees e ON c.customer_id = e.employee_id JOIN branch_locations bl ON e.branch_id = bl.branch_id WHERE a.open_date > DATE('now', '-6 months') GROUP BY account_type, bl.state LIMIT 10
Answer
It seems like the tool call was successful, but you didn't provide the result. Please provide the result of the tool call, and I'll be happy to help you further.


In [18]:
response['output']

"It seems like the tool call was successful, but you didn't provide the result. Please provide the result of the tool call, and I'll be happy to help you further."